In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. 
            Answer questions using only the following context. Use the context as much as possible.
            If you don't know the answer just say you don't know, don't make it up:\n\n{context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [2]:
invoke_chain("Is Aaronson guilty?")


content='According to the document, Jones, Aaronson, and Rutherford were considered guilty of the crimes they were charged with.'


In [3]:
invoke_chain("What message did he write in the table?")


content='He wrote "2+2=5" in the dust on the table.'


In [4]:
invoke_chain("Who is Julia?")


content='Julia is a character who was involved with the protagonist in the story. They shared a relationship and had moments of connection and rebellion against the Party together.'


In [5]:
invoke_chain("Is Aaronson guilty?")


content='According to the document, Jones, Aaronson, and Rutherford were considered guilty of the crimes they were charged with.'
